In [3]:
!pip install torch torchvision tqdm h5py


In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # For progress bar
import time
import os
from PIL import Image
import h5py

In [5]:
# Custom Dataset for PlantVillage with added error handling for debugging
class PlantVillageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.skipped_files = 0  # Count how many files are skipped

        for subdir, _, files in os.walk(root_dir):
            print(f"Directory: {subdir}, Number of files: {len(files)}")
            print("First 5 files:", files[:5])  # Print first 5 files in each directory
            
            for file in files:
                file_path = os.path.join(subdir, file)
                try:
                    # Test if the file can be opened as an image (ignore file extension)
                    with Image.open(file_path) as img:
                        self.image_paths.append(file_path)
                except Exception as e:
                    # Log the error if the file cannot be opened as an image
                    print(f"Error opening file {file_path}: {e}")
                    self.skipped_files += 1

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [6]:
# Data augmentation for self-supervised learning
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [7]:
# Load PlantVillage dataset
root_dir = '/kaggle/input/plantvillage-dataset/PlantVillage_dataset_2' # Update with actual path
dataset = PlantVillageDataset(root_dir=root_dir, transform=train_transform)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=8)

Directory: /kaggle/input/plantvillage-dataset/PlantVillage_dataset_2, Number of files: 43460
First 5 files: ['64e034cb-54bb-4434-b49a-42efe260c966___CREC_HLB 7803.JPG', '0ade19e4-c48b-4e58-bddf-153d44d48c3e___Com.G_SpM_FL 9449.JPG', '919505a0-7e06-4a89-8faf-f308bd644a6a___RS_NLB 4141.JPG', 'f218fcff-fc71-4db2-9ed8-52d0871d5a67___CREC_HLB 7344.JPG', 'd08a1c48-3360-40e3-9c3d-e47c2812bed2___UF.GRC_YLCV_Lab 01920.JPG']


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
# Print dataset size
print(f"Total number of images in the dataset: {len(dataset)}")
# Calculate expected number of batches
expected_batches = len(dataset) // 64
print(f"Expected number of batches: {expected_batches}")

Total number of images in the dataset: 43460
Expected number of batches: 679


In [9]:
# Load MobileNet with ImageNet weights, removing the classifier for self-supervised learning
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.classifier = nn.Identity()


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 111MB/s] 


In [10]:
# Define MoCo-style Model with MobileNet as the encoder and dropout
class MoCoModelWithDropout(nn.Module):
    def __init__(self, base_encoder, dropout_prob=0.5):
        super(MoCoModelWithDropout, self).__init__()
        self.encoder_q = nn.Sequential(
            base_encoder,  # Query encoder
            nn.Dropout(p=dropout_prob)  # Add dropout layer
        )
        self.encoder_k = nn.Sequential(
            base_encoder,  # Key encoder
            nn.Dropout(p=dropout_prob)  # Add dropout layer
        )

    def forward(self, x_q, x_k):
        q = self.encoder_q(x_q)  # Encode the query with dropout
        k = self.encoder_k(x_k)  # Encode the key with dropout
        return q, k



In [11]:
# Initialize MoCo with MobileNet as the encoder and dropout
model_with_dropout = MoCoModelWithDropout(mobilenet, dropout_prob=0.5).cuda()

In [12]:
# Define the AdamW optimizer
optimizer = torch.optim.AdamW(model_with_dropout.parameters(), lr=0.01, weight_decay=0.01)


In [13]:
# Define the InfoNCE (contrastive) loss function with accuracy calculation
def contrastive_loss_with_accuracy(q, k, temperature=0.07):
    # Normalize the queries and keys
    q = nn.functional.normalize(q, dim=1)
    k = nn.functional.normalize(k, dim=1)

    # Compute positive logits (similarity between q and k)
    pos_logits = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)

    # Compute negative logits (similarity between q and all keys)
    neg_logits = torch.mm(q, k.T)

    # Combine the logits and apply softmax
    logits = torch.cat([pos_logits, neg_logits], dim=1)
    logits /= temperature
    labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()

    # Compute InfoNCE loss
    loss = nn.CrossEntropyLoss()(logits, labels)

    # Calculate accuracy: Check if the highest logit is the positive one
    _, preds = torch.max(logits, dim=1)
    accuracy = (preds == labels).float().mean().item()

    return loss, accuracy

In [14]:
# Training loop for 100 epochs with accurate loss and accuracy calculation
num_epochs = 100
for epoch in range(num_epochs):
    model_with_dropout.train()
    total_loss = 0
    total_accuracy = 0
    start_time = time.time()
    
    # Progress bar for the current epoch
    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    
    for batch in progress_bar:
        x_q = batch.cuda()
        x_k = batch.cuda()  # Ideally, different augmentations should be applied here for MoCo

        # Forward pass through MoCo model
        q, k = model_with_dropout(x_q, x_k)

        # Calculate contrastive loss and accuracy
        loss, accuracy = contrastive_loss_with_accuracy(q, k)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy
        progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy * 100)
    
    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_accuracy / len(train_loader)
    elapsed_time = time.time() - start_time
    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy*100:.2f}%, Time: {elapsed_time:.2f}s")

Epoch [1/100]: 100%|██████████| 680/680 [05:22<00:00,  2.11it/s, accuracy=75, loss=0.72]   


Epoch [1/100], Avg Loss: 1.1294, Accuracy: 71.18%, Time: 322.01s


Epoch [2/100]: 100%|██████████| 680/680 [05:19<00:00,  2.13it/s, accuracy=50, loss=0.702]  


Epoch [2/100], Avg Loss: 1.0417, Accuracy: 72.88%, Time: 319.44s


Epoch [3/100]: 100%|██████████| 680/680 [05:18<00:00,  2.13it/s, accuracy=100, loss=0.697] 


...

Epoch [98/100], Avg Loss: 0.9042, Accuracy: 76.90%, Time: 312.54s


Epoch [99/100]: 100%|██████████| 680/680 [05:12<00:00,  2.18it/s, accuracy=25, loss=0.702]  


Epoch [99/100], Avg Loss: 0.9159, Accuracy: 76.07%, Time: 312.44s


Epoch [100/100]: 100%|██████████| 680/680 [05:12<00:00,  2.18it/s, accuracy=75, loss=0.695]  

Epoch [100/100], Avg Loss: 0.9015, Accuracy: 76.91%, Time: 312.36s


In [21]:
# Saving the model in different formats after all epochs
def save_model(model):
    torch.save(model.state_dict(), "MOCO_model_weights_final.pth")
    torch.save(model, "MOCO_full_model_final.pth")
    with h5py.File("MOCO_model_weights_final.h5", 'w') as f:
        for name, param in model.named_parameters():
            f.create_dataset(name, data=param.cpu().detach().numpy())

# Save model after training completes
save_model(model_with_dropout)

print("model saved")

model saved
